# Projections
*Arthur Ryman, lasted updated 2025-09-01*

## Introduction

The goal of this notebook is to analyze the current semantics of some classes that govern the mapping from model
space to scene space. 
My goal is to answer the following questions.

### Q1: Should Projection take a camera location as an init parameter?
The init parameters for Projection class are called scene_x, scene_y, and camera_z,
which makes them look dissimilar, 
but are they in fact components of a camera position vector in model space?

### Q2: Does PuzzleCube3D need the init parameter cube_centre?
The init parameters for PuzzleCube3D includes cube_centre. 
Does it really give additional expressibility or can its effect
be achieved by chaning the projection?

### Q3: Does Puzzle3D need the init parameter cube_one_centre?
Similarly, the init parameters for Puzzle3D include projection and cube_one_centre,
but can cube_one_centre be eliminated by chosing a different projection.

# The Projection Class

Source code: 
[projection.py](https://github.com/agryman/instant-insanity/blob/main/src/instant_insanity/core/projection.py)

Here is its init method:

In [10]:
from listings.list_python import lst
from instant_insanity.core.projection import Projection

lst(Projection.__init__)

   1     def __init__(self, scene_x: float = 0.0, scene_y: float = 0.0, camera_z: float = 2.0, scale: float = 1.0) 
   2         self.scene_x = scene_x                                                                                
   3         self.scene_y = scene_y                                                                                
   4         self.camera_z = camera_z                                                                              
   5         self.scale = scale                                                                                    
   6                                                                                                               
   7         scene_origin: Point3D = scene_x * RIGHT + scene_y * UP + camera_z * OUT                               
   8         scene_per_model: float = scale                                                                        
   9         self.conversion = ModelToSceneConversion(scene_origin, scene_per_model)                               
  10                                                                                                               

It is clear from the listing that the code is indeed treating the scene_x, scene_y, and camera_z as the components of a vector.
It remains to confirm that the projection does in fact map this vector to the origin of scene space.

### Using SymPy for Verification

The Projection class, and all other classes I have created for use with Manim, 
use NumPy to represent points in both model space and scene space.
I could therefore write a suite of NumPy test cases to verify that the scene_origin vector in model space 
does indeed get mapped to the origin of scene space.

However, that would only verify the behaviour in a finite number of cases and would not lead to a clearer understanding of the code.
Given that projections are fairly mathematical, it might be feasible to verify the behaviour in all cases by using 
[symbolic exection](https://en.wikipedia.org/wiki/Symbolic_execution) 
of the code.
Therefore, I am going to try using SymPy to verify the code symbolically.

## Symbolic Projection

The Project class requires that its inputs be NumPy arrays.
I have therefore created parallel SymPy versions of the code.
For example, here's the init method for the symbolic version of Projection.

In [11]:
import instant_insanity.core.symbolic_projection as sp

lst(sp.Projection.__init__)

   1     def __init__(self,                                                                                        
   2                  scene_x: Scalar = S.Zero,                                                                    
   3                  scene_y: Scalar = S.Zero,                                                                    
   4                  camera_z: Scalar = S.Zero,                                                                   
   5                  scale: Scalar = S.One) -> None:                                                              
   6         self.scene_x = scene_x                                                                                
   7         self.scene_y = scene_y                                                                                
   8         self.camera_z = camera_z                                                                              
   9         self.scale = scale                                                                                    
  10                                                                                                               
  11         scene_origin: Vector = scene_x * UNIT_I + scene_y * UNIT_J + camera_z * UNIT_K                        
  12         scene_per_model: Scalar = scale                                                                       
  13         self.conversion = ModelToSceneConversion(scene_origin, scene_per_model)                               
  14                                                                                                               

In [9]:
S.Zero.is_real

True

In [3]:
type Scalar = Expr
type Vector = Matrix

def scalar(name: str) -> Scalar:
    return symbols(name, real=True)

def vector(name: str) -> Vector:
    return Matrix(symbols(name + ':3', real=True))


s = scalar('s')
x = vector('x')
y = vector('y')

print(s, x, y, s * x, x + y)

s Matrix([[x0], [x1], [x2]]) Matrix([[y0], [y1], [y2]]) Matrix([[s*x0], [s*x1], [s*x2]]) Matrix([[x0 + y0], [x1 + y1], [x2 + y2]])


In [4]:
s * x

Matrix([
[s*x0],
[s*x1],
[s*x2]])

In [5]:
x + y

Matrix([
[x0 + y0],
[x1 + y1],
[x2 + y2]])

In [7]:
import instant_insanity.core.symbolic_projection as sp

NameError: name 'np' is not defined

In [ ]:
lst(sp.ModelToSceneConversion.convert_model_to_scene)

In [ ]:
scene_per_model = scalar('alpha')
scene_per_model

In [ ]:
scene_origin = vector('S')
scene_origin

In [ ]:
model_point = vector('M')
model_point

In [ ]:
model_point.shape

In [ ]:
scene_per_model.is_real

In [ ]:
[p.is_real for p in model_point]

In [ ]:
conversion = ModelToSceneConversion(scene_origin, scene_per_model)
conversion

In [ ]:
scene_point = conversion.convert_model_to_scene(model_point)
scene_point

In [ ]:
model_point_2 = conversion.convert_scene_to_model(scene_point)
model_point_2

In [ ]:
simplify(model_point - model_point_2)

In [ ]:
scene_point_2 = conversion.convert_model_to_scene(model_point_2)
scene_point_2

In [ ]:
scene_point

In [ ]:
scene_point_2 - scene_point

In [ ]:
from instant_insanity.core.projection import ModelToSceneConversion

In [ ]:
lst(ModelToSceneConversion.convert_model_to_scene)

In [ ]:
lst(ModelToSceneConversion.convert_scene_to_model)

In [ ]:
m_s = ModelToSceneConversion(scene_origin, scene_per_model)

m_s.scene_origin

In [ ]:
m_s.scene_per_model

In [ ]:
import numpy as np


In [ ]:
np.isclose(scene_per_model, 0.0)

In [ ]:
symbols('theta1:23')[4]